# Data Vis: Client-Side Interactivity
* Notebook 1: Introduction to Plotly

## Setup

In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Plotly

__Plotly__ (https://plotly.com/python/) offers two primary interfaces:

* __Plotly Express (px)__: A high-level, declarative API designed for rapid, concise creation of common chart types with minimal code. Leanr more at https://plotly.com/python/plotly-express/.

* __Plotly Graph Objects (go)__: A lower-level, object-oriented API that exposes the full power of Plotly’s figure schema, enabling fine-grained control and customization. Learn more at https://plotly.com/python/graph-objects/.

Below, we will primarily use Plotly Express for its simplicity and ease of use, but we will also blend in some Plotly Graph Objects for more advanced customization. 

## Data

In this notebook, we will use the `NYC Flights 13` dataset you already knwo from one of the previous notebook. We will use the joined dataset with all information about flights, airport, airlines, planes, and weather.


In [34]:
data = pd.read_csv("flights_joined.csv")

To speed up plotting, we will use a subset of the data. We will only use 10000 random rows of the dataset.

In [35]:
data = data.sample(n=10000, random_state=42)

In [36]:
data.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tzone_dest,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib
159280,2013,3,25,1929.0,1905,24.0,2236.0,2217,19.0,UA,...,America/New_York,37.94,35.06,89.25,320.0,9.20624,NaN,0.02,999.8,6.0
189101,2013,4,26,956.0,1000,-4.0,1257.0,1334,-37.0,DL,...,America/Los_Angeles,60.08,33.08,36.04,330.0,8.05546,16.11092,0.00,1029.2,10.0
212435,2013,5,21,1320.0,1309,11.0,1430.0,1414,16.0,EV,...,America/New_York,86.00,68.00,55.04,240.0,11.50780,NaN,0.00,1012.2,10.0
266804,2013,7,18,1222.0,1230,-8.0,1357.0,1419,-22.0,EV,...,America/New_York,98.06,69.08,39.22,300.0,9.20624,17.26170,0.00,1016.7,10.0
306581,2013,8,29,540.0,545,-5.0,921.0,921,0.0,B6,...,NaN,71.96,68.00,87.35,100.0,8.05546,NaN,0.00,1014.0,10.0


## Simple Interactive Plot

Let's use `plotly.express` to create a simple interactive plot. We will create a `scatter` plot of `dep_delay` vs. `arr_delay`.

In [37]:
fig = px.scatter(data, x="arr_delay", y="dep_delay")
fig.show()

You can also disable the toolbar and/or turn off all interactivity.

In [38]:
fig.show(config={"displayModeBar": False, "staticPlot": True})

Let's make the plot a bit nicer. First, we will set the title and axis labels. We can do this dircetly in `px.scatter` method. Not all layout options are available in functions like px.scatter, but we can always use the powerful `update_layout` method to customize a plot.

In [39]:
fig = px.scatter(data, x="arr_delay", y="dep_delay",
                 title="Arrival Delay vs Departure Delay",
                 labels={"arr_delay": "Arrival Delay", "dep_delay": "Departure Delay"})

fig.update_layout(width=800, height=800,
                  xaxis=dict(range=[-60, 500]),
                  yaxis=dict(range=[-60, 500]))
                  
fig.show()

Next, we use `add_trace` from `plotly.graph_objects` to manually add a 45-degree line to the plot. Let us first have a look at what output `go.Scatter`produces:

In [40]:
go.Scatter(x=[-60, 500], y=[-60, 500], mode='lines', line=dict(color='red')).to_json()

'{"line":{"color":"red"},"mode":"lines","x":[-60,500],"y":[-60,500],"type":"scatter"}'

The JSON dictionary shown above is what the Plotly Python library passes to the Plotly JavaScript library to render the plot. It contains all the information about the data, layout, and configuration of the plot. Belwo, we use this dictionary as input to the `add_trace` method to add a new trace to the plot.

In [41]:
fig = px.scatter(data, x="arr_delay", y="dep_delay", opacity=0.3,
                 title="Arrival Delay vs Departure Delay",
                 labels={"arr_delay": "Arrival Delay", "dep_delay": "Departure Delay"})

fig.add_trace(go.Scatter(x=[-60, 500], y=[-60, 500], mode='lines', line=dict(color='red')))

fig.update_layout(width=800, height=800,
                  xaxis=dict(range=[-60, 500]),
                  yaxis=dict(range=[-60, 500]),
                  showlegend=False)

fig.show()


## Adding Hover Information

There are many ways to customize the hover (mouse over) information in plotly. Let's experiment with the `hovermode` parameter. By default, it is set to `closest`. But we can set it to `x` or `y` and combine it with `unified` to create a highlight on the x or y axis, respectively. 

Alternatively, you can use the `showspikes` parameter of the x and y axis to customize the hover information:
* fig.update_xaxes(showspikes=True)
* fig.update_yaxes(showspikes=True)

In [42]:
fig = px.scatter(data, x="arr_delay", y="dep_delay", opacity=0.3,
                 title="Arrival Delay vs Departure Delay",
                 labels={"arr_delay": "Arrival Delay", "dep_delay": "Departure Delay"})

fig.add_trace(go.Scatter(x=[-60, 500], y=[-60, 500], mode='lines', line=dict(color='red')))

fig.update_layout(width=800, height=800,
                  xaxis=dict(range=[-60, 500]),
                  yaxis=dict(range=[-60, 500]),
                  showlegend=False,
                  hovermode="x unified")

fig.show()


We can also modify the text inside the hover box. By default, plotly produces a hover box with those columns that are used in the plot. We can customize the hover box by using the `hover_data` parameter to add additional columns to the hover box. We can also use the `hover_name` parameter to set a column that will be used as the main label in the hover box. Note that using unified hover mode will overwrite the styling of the hover box.

In [43]:
fig = px.scatter(data, x="arr_delay", y="dep_delay", opacity=0.3,
                 title="Arrival Delay vs Departure Delay",
                 labels={"arr_delay": "Arrival Delay", "dep_delay": "Departure Delay"},
                 hover_data=["origin", "dest"],
                 hover_name="flight")

fig.add_trace(go.Scatter(x=[-60, 500], y=[-60, 500], mode='lines', line=dict(color='red')))

fig.update_layout(width=800, height=800,
                  xaxis=dict(range=[-60, 500]),
                  yaxis=dict(range=[-60, 500]),
                  showlegend=False)

fig.show()


We can also use the `hovertemplate` parameter to customize the hover box using HTML. Note how we can use the `customdata` parameter to pass additional variables from `hover_data` to the hover box.

In [44]:
fig = px.scatter(data, x="arr_delay", y="dep_delay", opacity=0.3,
                 title="Arrival Delay vs Departure Delay",
                 labels={"arr_delay": "Arrival Delay", "dep_delay": "Departure Delay"},
                 hover_data=["carrier", "flight", "origin", "dest"])

hovertemplate = (
    "<b>Flight %{customdata[0]}%{customdata[1]} from %{customdata[2]} to %{customdata[3]}</b><br>"
    "<b>Arrival Delay:</b> %{x} minutes<br>"
    "<b>Departure Delay:</b> %{y} minutes"
)
fig.update_traces(hovertemplate=hovertemplate)

fig.add_trace(go.Scatter(x=[-60, 500], y=[-60, 500], mode='lines', line=dict(color='red')))

fig.update_layout(width=800, height=800,
                  xaxis=dict(range=[-60, 500]),
                  yaxis=dict(range=[-60, 500]),
                  showlegend=False)

fig.show()


## Your Turn (15 minutes)

Now it's your turn. Create an interactive scatterplot with arrival delay on the y-axis and a weather-related variable on the x-axis. Customize the hover information to include relevant details and style the popup. 

In [45]:
# YOUR CODE HERE

## Adding Buttons and Dropdowns

Custom buttons and dropdowns can be added to the plot to control the data or layout of the plot. For examples, all customizations done with `update_layout` or `update_traces` can be controlled with buttons or dropdowns.

In [46]:
from nycflights13 import weather
weather_march_all = weather[(weather['month'] == 3)]
weather_march_jfk = weather[(weather['month'] == 3) & (weather['origin'] == 'JFK')]
weather_march_lga = weather[(weather['month'] == 3) & (weather['origin'] == 'LGA')]
weather_march_ewr = weather[(weather['month'] == 3) & (weather['origin'] == 'EWR')]

We first use buttons to toggle the plot type. We will allow the user to switch between a line chart and a point chart, both showing a time series of temperatures in New York City in March.

In [47]:
fig = px.line(weather_march_jfk, x="time_hour", y="temp")

my_buttons = [
    {
        "label": "Line Chart",
        "method": "update",
        "args": [
            {"type": "scatter", "mode": "lines"},
            {
                "yaxis": {"range": [20, 60]},
                "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}
            }
        ]
    },
    {
        "label": "Point Chart",
        "method": "update",
        "args": [
            {"type": "scatter", "mode": "markers"},
            {
                "yaxis": {"range": [20, 60]},
                "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}
            }
        ]
    }
]

fig.update_layout(
    title="Temperature at JFK in March 2013",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
    updatemenus=[{
        "type": "buttons",
        "buttons": my_buttons,
        "direction": "left",
        "active": 0,
        "showactive": True,
        "x": 1.0,
        "y": 1.15,
        "xanchor": "right",
        "yanchor": "top"
    }]
)

fig.show()

In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_march_jfk["time_hour"], y=weather_march_jfk["temp"], mode='lines', name="JFK"))
fig.add_trace(go.Scatter(x=weather_march_lga["time_hour"], y=weather_march_lga["temp"], mode='lines', name="LGA"))
fig.add_trace(go.Scatter(x=weather_march_ewr["time_hour"], y=weather_march_ewr["temp"], mode='lines', name="EWR"))

my_dropdown_buttons = [
    {
        "label": "All Airports",
        "method": "update",
        "args": [{"visible": [True, True, True]},
                 {"yaxis": {"range": [20, 60]},
                  "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}}]
    },
    {
        "label": "JFK",
        "method": "update",
        "args": [{"visible": [True, False, False]},
                 {"yaxis": {"range": [20, 60]},
                  "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}}]
    },
    {
        "label": "LGA",
        "method": "update",
        "args": [{"visible": [False, True, False]},
                 {"yaxis": {"range": [20, 60]},
                  "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}}]
    },
    {
        "label": "EWR",
        "method": "update",
        "args": [{"visible": [False, False, True]},
                 {"yaxis": {"range": [20, 60]},
                  "xaxis": {"range": ["2013-03-01T00:00:00Z", "2013-03-31T23:59:59Z"]}}]
    }
]

fig.update_layout(
    title="Temperature at NYC Airports in March 2013",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
    updatemenus=[{
        "type": "dropdown",
        "buttons": my_dropdown_buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.0,
        "y": 1.15,
        "xanchor": "right",
        "yanchor": "top"
    }]
)

fig.show()

Especially for time series data, we can add buttons with a `rangeselector` to the x-axis to allow the user to zoom in on a specific time range. This is especially useful for large datasets with many data points.

In [51]:
fig = px.line(weather_march_jfk, x="time_hour", y="temp")

date_buttons = [
    {'count': 3, 'step': 'day', 'label': '3 Days', 'stepmode': 'backward'},
    {'count': 7, 'step': 'day', 'label': '7 Days', 'stepmode': 'backward'},
    {'count': 14, 'step': 'day', 'label': '14 Days', 'stepmode': 'backward'},
    {'step': 'all'}
]

fig.update_layout(
    title="Temperature at JFK in March 2013",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
    xaxis={'rangeselector': {'buttons': date_buttons}},
)

fig.show()

In addition, or alternatively, we can add a `rangeslider` to the x-axis. This allows the user to zoom in on a specific time range by dragging the slider.

In [53]:
fig = px.line(weather_march_jfk, x="time_hour", y="temp")

date_buttons = [
    {'count': 3, 'step': 'day', 'label': '3 Days', 'stepmode': 'backward'},
    {'count': 7, 'step': 'day', 'label': '7 Days', 'stepmode': 'backward'},
    {'count': 14, 'step': 'day', 'label': '14 Days', 'stepmode': 'backward'},
    {'step': 'all'}
]

fig.update_layout(
    title="Temperature at JFK in March 2013",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
    xaxis={'rangeselector': {'buttons': date_buttons}, 
           'rangeslider': {'visible': True}},
)

fig.show()

## Your Turn (30 minutes)

Now it's your turn. Create an interactive time series plot of delay times using the controls we just learned. For example, add buttons to toggle between different plot types, different time series, or different time ranges.

In [ ]:
# YOUR CODE HERE